In [1]:
import pandas as pd
import pg8000

In [2]:
def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

In [3]:
query = "SELECT cell.name AS cell_name, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%subthreshold%%' THEN 1 ELSE NULL END) AS subthreshold, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1LSFINEST150112%%' THEN 1 ELSE NULL END) AS longsquare, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1SSFINEST150112%%' THEN 1 ELSE NULL END) AS shortsquare, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1RP25PR1S141203%%' THEN 1 ELSE NULL END) AS ramp, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%Blip%%' THEN 1 ELSE NULL END) AS cap_check, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C2CHIRP171103%%' THEN 1 ELSE NULL END) AS chirp, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1NSD1SHORT17110%%' THEN 1 ELSE NULL END) AS noise_1, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1NSD2SHORT17110%%' THEN 1 ELSE NULL END) AS noise_2, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C2SSTRIPLE171103%%' THEN 1 ELSE NULL END) AS triple \
FROM specimens cell JOIN ephys_sweeps ess ON cell.id = ess.specimen_id \
JOIN ephys_stimuli ON ess.ephys_stimulus_id = ephys_stimuli.id \
WHERE cell.patched_cell_container NOTNULL \
GROUP BY cell.name"
df2 = get_lims_dataframe(query)
df2.head()

,noise_1,ramp,shortsquare,chirp,noise_2,subthreshold,cap_check,longsquare,triple,cell_name
0,0,2,5,0,0,3,1,4,0,Oxtr-T2A-Cre;Ai14-351471.04.01.01
1,0,0,0,0,0,0,0,4,0,Slc32a1-IRES-Cre;Ai14-326812.04.02.05
2,0,1,3,0,0,0,0,1,0,Slc32a1-IRES-Cre;Ai14-305535.06.02.01
3,0,3,7,0,0,0,0,0,0,Gad2-IRES-Cre;Ai14-267341.07.02.02
4,0,1,3,0,0,0,0,0,0,Slc17a6-IRES-Cre;Ai14-309388.03.01.01


In [196]:
query = "SELECT * FROM ephys_sweeps WHERE specimen_id = 653863491"
df1 = get_lims_dataframe(query)
df1.head()

Could not find results for your query.


""


In [4]:
df2.to_csv('sweep_count.csv')

In [9]:
lims_query = "SELECT cell.name, proj.code, err.id, f.storage_directory \
FROM specimens cell JOIN ephys_sweeps ess ON cell.id = ess.specimen_id \
JOIN ephys_stimuli ON ess.ephys_stimulus_id = ephys_stimuli.id \
JOIN projects proj ON cell.project_id = proj.id \
JOIN ephys_roi_results err ON cell.ephys_roi_result_id = err.id \
JOIN well_known_files f ON f.attachable_type = 'EphysRoiResult' AND f.attachable_id = err.id \
WHERE cell.patched_cell_container NOTNULL AND\
(err.failed_bad_rs = 'FALSE' \
AND err.failed_electrode_0 = 'FALSE' \
AND err.failed_no_seal = 'FALSE') \
GROUP BY cell.name, proj.code, err.id, f.storage_directory"

df3 = get_lims_dataframe(lims_query)
df3.head()

,storage_directory,code,name,id
0,/allen/programs/celltypes/production/mousecell...,T301x,Vip-IRES-Cre;Ai14-336703.04.02.01,604626206
1,/allen/programs/celltypes/production/mousecell...,T301x,Rbp4-Cre_KL100;Ai14-335444.05.02.03,603515658
2,/allen/programs/celltypes/production/mousecell...,mIVSCC-MET,Rorb-IRES2-Cre;Ai14-355893.03.01.02,644881787
3,/allen/programs/celltypes/production/humancell...,H301x,H17.03.007.12.14.04,592479431
4,/allen/programs/celltypes/production/mousecell...,mIVSCC-MET,Ndnf-IRES2-dgCre;Ai14-377313.05.02.01,667493807


In [13]:
df = pd.merge(left = df2, right = df3, left_on = 'cell_name', right_on = 'name', how = 'right')

In [14]:
df.head()

,noise_1,ramp,shortsquare,chirp,noise_2,subthreshold,cap_check,longsquare,triple,cell_name,storage_directory,code,name,id
0,0,2,5,0,0,3,1,4,0,Oxtr-T2A-Cre;Ai14-351471.04.01.01,/allen/programs/celltypes/production/mousecell...,mIVSCC-MET,Oxtr-T2A-Cre;Ai14-351471.04.01.01,640054173
1,0,1,3,0,0,0,0,0,0,Rorb-IRES2-Cre;Ai14-315894.04.02.02,/allen/programs/celltypes/production/mousecell...,T301x,Rorb-IRES2-Cre;Ai14-315894.04.02.02,584880187
2,0,1,1,0,0,5,1,4,0,Htr3a-Cre_NO152;Ai14-361253.03.02.01,/allen/programs/celltypes/production/mousecell...,mIVSCC-MET,Htr3a-Cre_NO152;Ai14-361253.03.02.01,652959147
3,0,2,11,0,0,0,0,9,0,H17.06.012.14.11.02,/allen/programs/celltypes/production/humancell...,H301,H17.06.012.14.11.02,611822740
4,0,2,8,0,0,0,0,0,0,Vip-IRES-Cre;Ai14-342919.04.02.01,/allen/programs/celltypes/production/mousecell...,T301x,Vip-IRES-Cre;Ai14-342919.04.02.01,614061137


In [15]:
df.to_csv('sweep_count.csv')